<a href="https://colab.research.google.com/github/IlyaZutler/GBR-agents/blob/main/GBR_Agents_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import random
import numpy as np
!pip install simpy
import simpy

In [ ]:
class Agent:
    def __init__(self, env, location, speed):
        self.env = env
        self.location = location
        self.base_location = location  # Место дислокации
        self.speed = speed
        self.status = 'Free'
        self.total_busy_time = 0
        self.handled_alarms = 0
        self.action = env.process(self.run())

    def run(self):
        while True:
            if self.status == 'Busy':
                yield self.env.timeout(1)
            else:
                yield self.env.timeout(1)
                self.status = 'Free'

    def travel_time(self, destination):
        return abs(self.location - destination) / self.speed

    def handle_alarm(self, alarm):
        self.status = 'Busy'
        travel_time = self.travel_time(alarm.location)
        yield self.env.timeout(travel_time)
        yield self.env.timeout(alarm.complexity)
        self.total_busy_time += travel_time + alarm.complexity
        self.handled_alarms += 1
        self.location = alarm.location

        # Возвращение к месту дислокации
        return_time = self.travel_time(self.base_location)
        yield self.env.timeout(return_time)
        self.location = self.base_location
        self.status = 'Free'

class Alarm:
    def __init__(self, env, location, complexity):
        self.env = env
        self.location = location
        self.complexity = complexity

def generate_alarms(env, agents, interval):
    while True:
        yield env.timeout(random.expovariate(1.0 / interval))
        location = random.uniform(0, 1)
        complexity = np.random.lognormal(mean=0, sigma=1)
        alarm = Alarm(env, location, complexity)

        # Найти ближайшего свободного агента
        free_agents = [agent for agent in agents if agent.status == 'Free']
        if free_agents:
            nearest_agent = min(free_agents, key=lambda agent: agent.travel_time(alarm.location))
            env.process(nearest_agent.handle_alarm(alarm))
        else:
            print(f"No available agents at time {env.now}")

def monitor(env, agents, results):
    while True:
        yield env.timeout(1)
        busy_agents = sum(agent.status == 'Busy' for agent in agents)
        results['busy_agents'].append(busy_agents / len(agents))

if __name__ == "__main__":
    random.seed(42)
    np.random.seed(42)

    env = simpy.Environment()
    agents = [Agent(env, location=i/10, speed=1) for i in range(10)]
    interval = 10  # Средний интервал между тревогами

    results = {'total_alarms': 0, 'average_response_time': 0, 'alarms_above_threshold': 0, 'busy_agents': []}

    env.process(generate_alarms(env, agents, interval))
    env.process(monitor(env, agents, results))

    env.run(until=100)

    # Подсчет итоговых результатов
    results['total_alarms'] = sum(agent.handled_alarms for agent in agents)
    total_response_time = sum(agent.total_busy_time for agent in agents)
    results['average_response_time'] = total_response_time / results['total_alarms'] if results['total_alarms'] > 0 else 0
    threshold = 5
    results['alarms_above_threshold'] = sum(
        1 for agent in agents for _ in range(agent.handled_alarms)
        if agent.total_busy_time / agent.handled_alarms > threshold
    )

    print("Total alarms:", results['total_alarms'])
    print("Average response time:", results['average_response_time'])
    print("Alarms above threshold:", results['alarms_above_threshold'])
    print("Busy agents over time:", np.mean(results['busy_agents']))
    for i, agent in enumerate(agents):
        print(f"Agent {i} handled {agent.handled_alarms} alarms, busy time: {agent.total_busy_time}")